In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import pickle
from itertools import combinations
from scipy import stats

In [2]:
#read in decoding accuracy scores

#set up directory to read in decoding scores
subjdate = '20211119'
S = 'S03'
rootdir = '/misc/data12/sjapee/Prosopagnosia-Emotion/MEG'
data_dir = rootdir + '/MEGData/' + subjdate
out_dir = data_dir + '/plots'
in_filepath = data_dir + '/' + S + '_IMbased_allscores.pkl'

#load scores into all_scores, a dictionary of the following format: 
# key: tuple of two trial codes (1,2) representing a pair of images
# value: a 1x1081 list of decoding scores for each timepoint
in_file = open(in_filepath,'rb')
all_scores = pickle.load(in_file)
in_file.close()

In [3]:
%matplotlib qt
#convert the dictionary to a 3D numpy array (MEG RDM)
#each key dictates the row and column number in the array
#only fills in the upper triangle, off diagonal, remaining entries = 0

nrows = 125
nsamples = len(all_scores[1,2])

rdm_meg = np.zeros((nrows,nrows,nsamples))

for key,value in all_scores.items():

    row = key[0] - 1
    col = key[1] - 1

    rdm_meg[row,col] = value

print(rdm_meg[0,124,0])


plt.imshow(rdm_meg[:,:,540])
plt.title('MEG RDM at timepoint 540 (~350 ms)')
plt.colorbar()



0.65


In [4]:
#helper function for building model rdms
def build_trial_key(trialcode_func):
    #takes in one of the three functions below and returns a dict mapping each trial code to its corresponding id, expression, or view code
    key = {}
    for i in range(5):
        codes = (trialcode_func(i+1))
        #print(codes)
        
        for code in codes:
            key[code] = i+1

    return key


#these are the three trial code functions used as arguments (trialcode_func) for other helper functions build_trial_key and build_model_rdm

#sequences of numbers produces are based on the trial code structure of the stimuli (5 actors x 5 expr x 5 views)
def get_trials_by_expression(exp_code):
    
    trial_codes = []
    start_code = (exp_code*5) - 4
    
    for i in range(start_code,125,25):
        for j in range(i,i+5):
            trial_codes.append(j)
    return trial_codes

def get_trials_by_id(actor_code):
    start_code = (actor_code * 25) - 24
    return [i for i in range(start_code,start_code+25)]

def get_trials_by_view(view_code):
    return [i for i in range(view_code, 126,5)]

#print(build_trial_key(get_trials_by_view))

In [5]:
#build a model rdm for each condition

def build_model_rdm(trialcode_func):

    #build the key linking trial codes to exp/id/view codes
    key = build_trial_key(trialcode_func)

    n = len(key)

    #initialize an empty numpy array
    rdm = np.full((n,n),np.nan)

    #iterate through each row and column number to build the model rdm
    for r in range(n):

        for c in range(n):

            #if the two trial codes map to the same expression, id or view, fill this entry with 0
            if key[r+1] == key[c+1]:
                rdm[r,c] = 0
            #if different, fill this entry with a value of 1
            else:
                rdm[r,c] = 1

    return rdm

rdm_view = build_model_rdm(get_trials_by_view)
rdm_id = build_model_rdm(get_trials_by_id)
rdm_exp = build_model_rdm(get_trials_by_expression)


In [44]:
print(rdm_meg[:,:,1].shape)
print(len(rdm_meg[0,0]))

(125, 125)
1081


In [6]:
#correlate one model rdm with every timepoint of the meg rdm
#return a 1x1081 matrix: timecourse of Spearman correlations
def rsa(model, meg):

    timecourse = []

    #extract the upper triangle from the model rdm and flatten it, mask values of 0
    model_flat = ma.masked_equal(np.ndarray.flatten(np.triu(model,k=1)),0)

    #iterate through each timepoint
    for i in range(len(meg[0,0])): # 

        #extract the meg rdm for that timepoint
        slice = meg[:,:,i]

        slice_flat = ma.masked_equal(np.ndarray.flatten(slice),0)
        #print(model_flat.mean())
        #print(slice_flat.mean())
        correlation,p = stats.spearmanr(model_flat,slice_flat)

        timecourse.append(correlation)

    return timecourse


rsa_view = rsa(rdm_view,rdm_meg)
rsa_id = rsa(rdm_id,rdm_meg)
rsa_exp = rsa(rdm_exp,rdm_meg)
print(len(rsa_id))
#print(rsa(rdm_id,rdm_meg))    



1081


In [7]:
#plot results of rsa
#set ranges for x and y axes
xmin = -0.1
xmax = 0.8
ymin = 0.42
ymax = 0.57
axes = [xmin,xmax,ymin,ymax]
nsamples = len(rsa_view)
xvals = np.linspace(xmin,xmax,nsamples)


plt.plot(xvals,rsa_exp,label='expression')
plt.plot(xvals,rsa_id,label='identity')
plt.plot(xvals,rsa_view,label='viewpoint')
#plt.axis(axes)
plt.title('Spearmans correlations from RSA')
plt.legend()